In [ ]:
import pandas as pd
import scanpy as sc

In [ ]:
# convert to Ensembl protein ID
links = pd.read_csv("../../data/STRING/9606.protein.physical.links.v11.5.txt", sep=" ")
links['protein1'] = links['protein1'].str.split("9606.", expand=True)[1]
links['protein2'] = links['protein2'].str.split("9606.", expand=True)[1]

In [ ]:
# convert Ensembl protein ID to Ensembl gene ID
eP2eG = pd.read_csv("../../data/Ensembl/EnsemblP2EnsemblG.txt", sep="\t", header=None).dropna().copy()
eP2eG = eP2eG.set_index(0)[1].to_dict()
links['protein1'] = links['protein1'].map(eP2eG)
links['protein2'] = links['protein2'].map(eP2eG)
links = links.dropna().copy()

In [ ]:
data = sc.read_h5ad("../../data/perturb/K562_gwps_normalized_bulk_01.h5ad")
id2name = data.var['gene_name'].to_dict()
links['protein1'] = links['protein1'].map(id2name)
links['protein2'] = links['protein2'].map(id2name)
# keep only both ends existed in the full perturb-seq data
links = links.dropna()
links = links.loc[:, ['protein1', 'protein2']].copy()

In [ ]:
# map to data index
gene = pd.read_csv("../../result/data/genes_gene_perturb", header=None)
gene_dict = gene.reset_index().set_index(0)['index'].to_dict()
links['protein1'] = links['protein1'].map(gene_dict)
links['protein2'] = links['protein2'].map(gene_dict)

In [ ]:
# remove nodes in full perturb-seq data (~9000) and keep only significant one (~2319)
links = links.dropna().astype(int)

In [ ]:
links = links.drop_duplicates()

In [ ]:
links.to_csv("../../result/network/STRING.tsv", sep="\t", header=False, index=False)